# Training GPT-J to replicate the Office Script


### Importing the Script
* From Kagle: https://www.kaggle.com/datasets/nasirkhalid24/the-office-us-complete-dialoguetranscript

### Standardizing the way that information is presented to the Model
* We need some standard format that will match the outputs of the vision team. We should probabily make a class for this with methods that can send the prompt to the model
### Importing the Model
* From HuggingFace (the transformers library), we will import GPT-J
* https://huggingface.co/EleutherAI/gpt-j-6B
### Tuning

## Attempt 2: Custom Transformer (probably will suck)

### Buidling the Model
### Pre-training
### Fine tuning

### General Imports

In [53]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
import os
import pandas as pd
import torch
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [19]:
!ls drive/MyDrive

 Brown		   'Copy of gnn_lab.ipynb'   released_lab_gnn
 CERTIFICATES	   'Final Project'	     The-Office-Lines-V4.csv
'Colab Notebooks'   released_lab	    'VEDANT SOP.docx'


In [26]:
data_file = "drive/Shareddrives/Final Project Data/The-Office-Lines-V4.csv"
data = pd.read_csv(data_file)
data = data.drop("Unnamed: 6", axis=1)

print(data[:5]) # Probably need to delete the last line

   season  episode  title  scene  speaker  \
0       1        1  Pilot      1  Michael   
1       1        1  Pilot      1      Jim   
2       1        1  Pilot      1  Michael   
3       1        1  Pilot      1      Jim   
4       1        1  Pilot      1  Michael   

                                                line  
0  All right Jim. Your quarterlies look very good...  
1         Oh, I told you. I couldn't close it. So...  
2  So you've come to the master for guidance? Is ...  
3         Actually, you called me in here, but yeah.  
4    All right. Well, let me show you how it's done.  


### Importing the Data

* Collect it by scene

In [37]:

## Keys of the dictionary will be scenes, Values will be a list of lines by each chracter
data_dictionary = {}

for index, row in data.iterrows():
  if row['scene'] not in data_dictionary:
    data_dictionary[row['scene']] = []

  data_dictionary[row['scene']].append(  ( row['speaker'], row['line'] ) )



In [38]:
data_dictionary[1]

[('Michael',
  'All right Jim. Your quarterlies look very good. How are things at the library?'),
 ('Jim', "Oh, I told you. I couldn't close it. So..."),
 ('Michael',
  "So you've come to the master for guidance? Is this what you're saying, grasshopper?"),
 ('Jim', 'Actually, you called me in here, but yeah.'),
 ('Michael', "All right. Well, let me show you how it's done.")]

### Standardizing Presentation of the TexT

Example of what we want to prompt the model with:

Prompt:

> Characters: Jim, Pam \\
> Objects in Scene: <What do we put here?> \\
> Lines:
>> Jim: Good morning, Pam \\
>> Pam:

Expected Text Completion:
> Characters: Jim, Pam \\
> Objects in Scene: <What do we put here?> \\
> Lines:
>> Jim: Good morning, Pam \\
>> Pam: **Good morning, Jim!**








'Characters: /n a'

In [110]:
# class scene():
#     def __init__(self, characters = []) -> None:
#         self.characters = characters

#     def to_text(self):
#         output = f"Characters: "


class PromptGenerator:
  def __init__(self, data):
    assert type(data) == dict
    self.data       = data ### Dictionary where keys of the dictionary will be scenes, Values will be a list of lines by each chracter
    self.scenes     = set(self.data.keys())
    self.num_scenes = len(self.scenes)

  def to_text(self):
    text = ""
    for scene in self.scenes:
      text += self.get_prompt_for_scene(scene)  + " \n " # TODO: append new line to sentence

    return text 

  def get_prompt_for_scene(self, scene_number):
    assert scene_number in self.data

    scene_data                              = self.data[scene_number]
    _, characters_string, lines_in_scene    = self.get_characters_in_scene_and_lines(scene_number)
    objects_in_scene                        = self.get_all_nouns_in_scene(lines_in_scene)


    prompt       = "Characters: " + characters_string  + " \n "   # TODO: append new line to sentence
    prompt      += "Objects in Scene: " + objects_in_scene  + " \n " # TODO: Figure out how to generate what objects to put in scene  # TODO: append new line to sentence
    prompt      += "Lines: " + " \n ".join( lines_in_scene ) 

    return prompt


  def get_characters_in_scene_and_lines(self, scene_number):
    scene_data  = self.data[scene_number]

    characters_list = []
    lines_in_scene      = []
    for line_and_character in scene_data:
      characters_list.append(line_and_character[0])
      lines_in_scene.append(line_and_character[0] + ": " + line_and_character[1]) # TODO: append new line to sentence

    characters_string = ", ".join(characters_list)

    return characters_list, characters_string, lines_in_scene

  def get_all_nouns_in_scene(self, text):
    if type(text) == list:
      text = " ".join(text)
    ## TODO: Think about this more and work on later 
    ## TODO: Remove names
    temp = [word for (word, pos) in nltk.pos_tag(nltk.word_tokenize(text)) if pos[0] == 'N' and (pos != 'NNP')]
    return " ".join( temp )

  


In [111]:
promptGenerator = PromptGenerator(data_dictionary)

In [112]:
promptGenerator.get_prompt_for_scene(1)

"Characters: Michael, Jim, Michael, Jim, Michael \n Objects in Scene: quarterlies things library master guidance grasshopper right \n Lines: Michael: All right Jim. Your quarterlies look very good. How are things at the library? \n Jim: Oh, I told you. I couldn't close it. So... \n Michael: So you've come to the master for guidance? Is this what you're saying, grasshopper? \n Jim: Actually, you called me in here, but yeah. \n Michael: All right. Well, let me show you how it's done."

In [52]:


#https://stackoverflow.com/questions/33587667/extracting-all-nouns-from-a-text-file-using-nltk


txt = """Natural language processing (NLP) is a field of computer science, artificial intelligence, and computational linguistics concerned with the interactions between computers and human (natural) languages."""
print([word for (word, pos) in nltk.pos_tag(nltk.word_tokenize(txt)) if pos[0] == 'N'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


['language', 'processing', 'NLP', 'field', 'computer', 'science', 'intelligence', 'linguistics', 'interactions', 'computers', 'languages']


### Importing the Model

In [ ]:
https://www.kaggle.com/datasets/nasirkhalid24/the-office-us-complete-dialoguetranscript